In [1]:
#importing packages

import random
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier 
from deap import creator, base, tools, algorithms
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.tree import _tree
from sklearn.preprocessing import LabelEncoder
import sys
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
from statistics import mean
from sklearn.model_selection import cross_val_score

In [2]:
#loading dataset 

print(os.listdir("../archive (1)"))

# Be careful with path Directory.
#reading dataset

dataset=pd.read_csv('../archive (1)/city_day.csv')
dataset.describe()
dataset.info()

['## importing packages.py', 'city_day.csv', 'city_hour.csv', 'Dataset', 'Final', 'FUZZY', 'GA', 'GA.ipynb', 'GA1.ipynb', 'Ruleset', 'RuleSet_Creation (2).ipynb', 'stations.csv', 'station_day.csv', 'station_hour.csv', 't2.arff', 'test1.csv', 'test1.py', 'test2.csv', 'test2.py', 'test3.py', 'test4.py', 'test5f.py', 'test6f.py', 'test7GA.py', 'Test8.py']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29531 entries, 0 to 29530
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        29531 non-null  object 
 1   Date        29531 non-null  object 
 2   PM2.5       24933 non-null  float64
 3   PM10        18391 non-null  float64
 4   NO          25949 non-null  float64
 5   NO2         25946 non-null  float64
 6   NOx         25346 non-null  float64
 7   NH3         19203 non-null  float64
 8   CO          27472 non-null  float64
 9   SO2         25677 non-null  float64
 10  O3          25509 non-null  float64
 11

In [3]:

#removing null
#print(aha,"------------>",len(aha))
dataset.replace([np.inf, -np.inf], np.nan, inplace=True)
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer = imputer.fit(dataset.iloc[:,2:14].values)
dataset.iloc[:,2:14] = imputer.transform(dataset.iloc[:,2:14].values)


#preprocessing

def AIR_QUALITY_INDEX(x):
    if x <= 50:
        return 1.0
    elif x <= 100:
        return 2.0
    elif x <= 200:
        return 3.0
    elif x <= 300:
        return 4.0
    elif x <= 400:
        return 5.0
    elif x > 400:
        return 6.0
    else:
        return 0


dataset["Checks"] = (dataset["PM2.5"] > 0).astype(int) + \
                (dataset["PM10"] > 0).astype(int) + \
                (dataset["SO2"] > 0).astype(int) + \
                (dataset["NOx"] > 0).astype(int) + \
                (dataset["NH3"] > 0).astype(int) + \
                (dataset["CO"] > 0).astype(int) + \
                (dataset["O3"] > 0).astype(int)

dataset["AQI_calculated"] = round(dataset[["PM2.5", "PM10", "SO2", "NOx",
                                 "NH3", "CO", "O3"]].max(axis = 1))
dataset.loc["dag"] = np.NaN
dataset.loc[dataset.Checks < 3, "AQI_calculated"] = np.NaN

dataset["AQI_bucket_calculated"] = dataset["AQI_calculated"].apply(lambda x: AIR_QUALITY_INDEX(x))
dataset[~dataset.AQI_calculated.isna()].head(13)
print("Each labels and total number of tuples accordingly")
dataset[~dataset.AQI_calculated.isna()].AQI_bucket_calculated.value_counts()


Each labels and total number of tuples accordingly


3.0    16618
2.0     6435
1.0     3169
4.0     2151
5.0      766
6.0      392
Name: AQI_bucket_calculated, dtype: int64

In [4]:
print(dataset['City'].unique())
df = dataset
dataset = dataset[dataset['City'] == 'Hyderabad']
dataset.info()


['Ahmedabad' 'Aizawl' 'Amaravati' 'Amritsar' 'Bengaluru' 'Bhopal'
 'Brajrajnagar' 'Chandigarh' 'Chennai' 'Coimbatore' 'Delhi' 'Ernakulam'
 'Gurugram' 'Guwahati' 'Hyderabad' 'Jaipur' 'Jorapokhar' 'Kochi' 'Kolkata'
 'Lucknow' 'Mumbai' 'Patna' 'Shillong' 'Talcher' 'Thiruvananthapuram'
 'Visakhapatnam' nan]
<class 'pandas.core.frame.DataFrame'>
Index: 2006 entries, 14581 to 16586
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   City                   2006 non-null   object 
 1   Date                   2006 non-null   object 
 2   PM2.5                  2006 non-null   float64
 3   PM10                   2006 non-null   float64
 4   NO                     2006 non-null   float64
 5   NO2                    2006 non-null   float64
 6   NOx                    2006 non-null   float64
 7   NH3                    2006 non-null   float64
 8   CO                     2006 non-null   float64
 9   SO2        

In [5]:

def avg(l):
    #this returns average among list element
    return (sum(l)/float(len(l)))


def getFitness(Ind, X, y):
    #returns the subset of features fitness
    if(Ind.count(0) != len(Ind)):
        # get i with value 0
        Column = [i for i in range(
            len(Ind)) if Ind[i] == 0]

        # get features subset
        parse = X.drop(X.columns[Column], axis=1)
        sub_features = pd.get_dummies(parse)

        # Classifier model is loaded 

        #clf = LogisticRegression()
        #clf = DecisionTreeClassifier(max_depth=10)
        clf = RandomForestRegressor()
        return (avg(cross_val_score(clf, sub_features, y, cv=5)),)
    else:
        return(0,)



In [6]:

def GA(X, y, n_population, n_generation):
    # Deap global variables are intiated
    # creating Ind
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Ind", list, fitness=creator.FitnessMax)

    # create toolbox and tournament is set
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("Ind", tools.initRepeat,
                     creator.Ind, toolbox.attr_bool, len(X.columns))
    toolbox.register("population", tools.initRepeat, list,
                     toolbox.Ind)
    toolbox.register("evaluate", getFitness, X=X, y=y)
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # initialize parameters
    pop = toolbox.population(n=n_population)
    hof = tools.HallOfFame(n_population * n_generation)
    stat_oper = tools.Statistics(lambda ind: ind.fitness.values)
    stat_oper.register("avg", np.mean)
    stat_oper.register("min", np.min)
    stat_oper.register("max", np.max)

    # genetic algorithm 
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2,
                                   ngen=n_generation, stats=stat_oper, halloffame=hof,
                                   verbose=True)

    # return hall of fame
    return hof



In [7]:

def bestInd(hof, X, y):
    #selecting best Individuals
    Max_Acc = 0.0
    for Ind in hof:
        #print(type(Ind.fitness.values[0]),Ind.fitness.values[0],"-.-.-.")
        if(Ind.fitness.values[0] > Max_Acc):
            Max_Acc = Ind.fitness.values
            _Ind = Ind

    _IndHeader = [list(X)[i] for i in range(
        len(_Ind)) if _Ind[i] == 1]
    return _Ind.fitness.values, _Ind, _IndHeader



In [8]:
#Dir, n_pop, n_gen = getArguments()
Dir = "../archive (1)/city_day.csv"
n_pop = 3  # population
n_gen = 10 #generation
y = dataset.iloc[:, -1]
X = dataset.iloc[:,2:14]


In [9]:

# all features accuracy is calculated for base
individual = [1 for i in range(len(X.columns))]
print("Accuracy Calucalted for all features: \t" +str(getFitness(individual, X, y)) + "\n")


Accuracy Calucalted for all features: 	(0.9440136902911025,)



In [10]:
#  genetic algorithm
hof = GA(X, y, n_pop, n_gen)
# select the best individual
accuracy, individual, header = bestInd(hof, X, y)
print('Best Accuracy: \t' + str(accuracy))
print('Number of Features in Subset: \t' + str(individual.count(1)))
print('Individual: \t\t' + str(individual))
print('Feature Subset\t: ' + str(header))



gen	nevals	avg     	min     	max     
0  	3     	0.485254	0.202849	0.813292
1  	2     	0.811698	0.810485	0.813292
2  	0     	0.812634	0.811319	0.813292
3  	3     	0.801692	0.792427	0.816545
4  	2     	0.85362 	0.797064	0.947251
5  	1     	0.946923	0.946267	0.947251
6  	1     	0.947184	0.94705 	0.947251
7  	1     	0.947352	0.947251	0.947553
8  	0     	0.947452	0.947251	0.947553
9  	3     	0.727538	0.293333	0.945052
10 	0     	0.945052	0.945052	0.945052
Best Accuracy: 	(0.9472513618140408,)
Number of Features in Subset: 	4
Individual: 		[1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]
Feature Subset	: ['PM2.5', 'PM10', 'NOx', 'Toluene']


In [11]:
#calculating accuracy for selected final sub set features
#df = pd.read_csv(Dir, sep=',')

# with feature subset

X = X.loc[:,header]
y = dataset.iloc[:, -1]
#genetic algorithm works with RandomForestRegressor were as final clf used here is DecisionTreeClassifier
clf =DecisionTreeClassifier(max_depth=5,random_state=1)
#cross validating
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10)
Accuracy = cross_val_score(clf, X, y, cv=cv)



C:\Users\HM-PC\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\HM-PC\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\HM-PC\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\HM-PC\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 3 members, which is less than

In [12]:
print("Overall Accuracy with Feature Subset: \t" + str(avg(Accuracy)) + "\n")
print("_________________________________")
print("Overall Optimal Feature selection Result:")
print("_________________________________")
for i in range(0,len(header)):
    print(header[i])

Overall Accuracy with Feature Subset: 	0.9893310945273636

_________________________________
Overall Optimal Feature selection Result:
_________________________________
PM2.5
PM10
NOx
Toluene


In [13]:

#Chennai specific attribute selection instead of city
l = df[df["City"] == 'Chennai']

y = l.iloc[:,-1]
X = l.iloc[:,2:14]
clfp =DecisionTreeClassifier(max_depth=5,random_state=1)
#cross validating
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2)
Accuracy_Chennai = cross_val_score(clfp, X, y, cv=cv)
print("--------->",avg(Accuracy_Chennai))

print(len(X),len(y))

#  genetic algorithm
hof = GA(X, y, n_pop, n_gen)
# select the best individual
accuracy_Chennai, individual_Chennai, header_Chennai = bestInd(hof, X, y)

print(len(X),len(y))
X = X.loc[:,header_Chennai]
#y = l.iloc[:, :-1]

print(len(X),len(y))
#genetic algorithm works with RandomForestRegressor were as final clf used here is DecisionTreeClassifier
clff =DecisionTreeClassifier(max_depth=5,random_state=1)
#cross validating
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2)
Accuracy_Chennai = cross_val_score(clff, X, y, cv=cv)


---------> 0.995023631840796
2009 2009
C:\Users\HM-PC\AppData\Local\Programs\Python\Python39\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\HM-PC\AppData\Local\Programs\Python\Python39\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Ind' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
gen	nevals	avg     	min     	max     
0  	3     	-13.9357	-29.8062	0.401397
1  	2     	-3.86871	-12.4024	0.407425
2  	2     	0.401052	0.388867	0.407316
3  	0     	0.407316	0.407316	0.407316
4  	1     	0.357059	0.256544	0.407316
5  	2     	0.406731	0.404015	0.408861
6  	1     	0.401939	0.389642	0.408861
7  	2     	0.392

In [17]:
print("Chennai Accuracy with Feature Subset: \t" + str(avg(Accuracy_Chennai)) + "\n")
print("_________________________________")
print("Chennai Optimal Feature selection Result:")
print("_________________________________")
for i in range(0,len(header_Chennai)):
    print(header_Chennai[i])

Chennai Accuracy with Feature Subset: 	0.9721218905472634

_________________________________
Chennai Optimal Feature selection Result:
_________________________________
PM10
NH3
Xylene


In [18]:

#Ahmedabad specific attribute selection instead of city
l = df[df["City"] == 'Ahmedabad']

y = l.iloc[:,-1]
X = l.iloc[:,2:14]
clfp =DecisionTreeClassifier(max_depth=5,random_state=1)
#cross validating
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2)
Accuracy_Ahmedabad = cross_val_score(clfp, X, y, cv=cv)
print("--------->",avg(Accuracy_Ahmedabad))


print(len(X),len(y))

#  genetic algorithm
hof = GA(X, y, n_pop, n_gen)
# select the best individual
accuracy_Ahmedabad, individual_Ahmedabad, header_Ahmedabad = bestInd(hof, X, y)

print(len(X),len(y))
X = X.loc[:,header_Ahmedabad]
#y = l.iloc[:, :-1]

print(len(X),len(y))
#genetic algorithm works with RandomForestRegressor were as final clf used here is DecisionTreeClassifier
clff =DecisionTreeClassifier(max_depth=5,random_state=1)
#cross validating
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2)


C:\Users\HM-PC\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\HM-PC\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\HM-PC\AppData\Local\Programs\Python\Python39\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\HM-PC\AppData\Local\Programs\Python\Python39\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Ind' has alrea

In [20]:
Accuracy_Ahmedabad = cross_val_score(clff, X, y, cv=cv)
print("Ahmedabad Accuracy with Feature Subset: \t" + str(avg(Accuracy_Ahmedabad)) + "\n")
print("_________________________________")
print("Ahmedabad Optimal Feature selection Result:")
print("_________________________________")
for i in range(0,len(header_Ahmedabad)):
    print(header_Ahmedabad[i])

Ahmedabad Accuracy with Feature Subset: 	0.9947748756218905

_________________________________
Ahmedabad Optimal Feature selection Result:
_________________________________
PM2.5
PM10
NO
NO2
NH3
CO
SO2
O3
Benzene
Toluene
C:\Users\HM-PC\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\HM-PC\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


In [24]:

#Delhi specific attribute selection instead of city
l = df[df["City"] == 'Delhi']

y = l.iloc[:,-1]
X = l.iloc[:,2:14]
clfp =DecisionTreeClassifier(max_depth=5,random_state=1)
#cross validating
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2)
Accuracy_Delhi = cross_val_score(clfp, X, y, cv=cv)
print("--------->",avg(Accuracy_Delhi))
print(len(X),len(y))

#  genetic algorithm
hof = GA(X, y, n_pop, n_gen)
# select the best individual
accuracy_Delhi, individual_Delhi, header_Delhi = bestInd(hof, X, y)

print(len(X),len(y))
X = X.loc[:,header_Delhi]
#y = l.iloc[:, :-1]

print(len(X),len(y))
#genetic algorithm works with RandomForestRegressor were as final clf used here is DecisionTreeClassifier
clff =DecisionTreeClassifier(max_depth=5,random_state=1)
#cross validating
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2)


---------> 0.9945236318407961
2009 2009
C:\Users\HM-PC\AppData\Local\Programs\Python\Python39\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\HM-PC\AppData\Local\Programs\Python\Python39\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Ind' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
gen	nevals	avg     	min     	max    
0  	3     	0.763735	0.346699	0.97581
1  	0     	0.973439	0.968696	0.97581
2  	1     	0.975911	0.97581 	0.976112
3  	0     	0.976112	0.976112	0.976112
4  	2     	0.976032	0.975887	0.976112
5  	0     	0.976112	0.976112	0.976112
6  	3     	0.976283	0.975938	0.976734
7  	3     	0.97662

In [25]:
Accuracy_Delhi = cross_val_score(clff, X, y, cv=cv)
print("Delhi Accuracy with Feature Subset: \t" + str(avg(Accuracy_Delhi)) + "\n")
print("_________________________________")
print("Delhi Optimal Feature selection Result:")
print("_________________________________")
for i in range(0,len(header_Delhi)):
    print(header_Delhi[i])

Delhi Accuracy with Feature Subset: 	0.9940248756218905

_________________________________
Delhi Optimal Feature selection Result:
_________________________________
PM2.5
PM10
NO
NO2
NOx
O3
